In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib import dates

import matplotlib
import statsmodels.tsa.api as tsa

%matplotlib inline

matplotlib.style.use("Solarize_Light2")

from pylab import rcParams
import statsmodels.api as sm

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://m.aussizzgroup.com/uploadedfiles/BL_56_09092017213550.jpg)aussizzgroup.com

In [ ]:
df = pd.read_csv("/kaggle/input/pakistan-engineers-data/PITC_Engineering.csv", low_memory=False)
print(df.shape)
df.head().style.set_properties(**{'background-color':'green',
                                     'color': 'white'})

In [ ]:
df.isnull().sum()

In [ ]:
#Code by Taikutsu https://www.kaggle.com/ravels1991/simple-eda/comments

g = sns.lmplot(x="Longitude", y="Latitude", data=df,
           fit_reg=False, scatter_kws={"s": 30}, hue='Province', height=10)
plt.title('Engineers Employment by Province')
plt.show()

In [ ]:
fig_px = px.scatter_mapbox(df, lat="Latitude", lon="Longitude",
                           hover_name="Province",
                           zoom=11, height=300)
fig_px.update_layout(mapbox_style="open-street-map",
                     margin={"r":0,"t":0,"l":0,"b":0})

fig_px.show()

In [ ]:
fig_px.update_traces(marker={"size": [10 for x in df]})

In [ ]:
y = df.loc[~df.ID.isnull()][['ID', 'License_no', 'PEC_No']]

y_log=np.log1p(y)

In [ ]:
#Code by Oniel Gracious https://www.kaggle.com/onielg/simplecatboostwithclassifierchains/notebook

#histogram and normal probability plot
from scipy import stats

from scipy.stats import norm
target=['License_no','PEC_No','ID']
for t in target:
    fig = plt.figure()
    sns.distplot(y_log[t], fit=norm);
    res = stats.probplot(y_log[t], plot=plt)

In [ ]:
fig = px.bar(df, 
             x='Employment_Start_Date', y='Firm_Category', color_discrete_sequence=['#FFD700'],
             title='Engineers Employment', text='Engineering_Degree')
fig.show()

In [ ]:
fig = px.bar(df, 
             x='Employment_End_Date', y='Engineering_Degree', color_discrete_sequence=['#DC143C'],
             title='Engineers Employment', text='Firms_City')
fig.show()

In [ ]:
#Save for the next time, since CONSTANT didn't work here.

# imputing with a constant

#from sklearn.impute import SimpleImputer
#train_constant = train.copy()
#setting strategy to 'constant' 
#mean_imputer = SimpleImputer(strategy='constant') # imputing using constant value
#train_constant.iloc[:,:] = mean_imputer.fit_transform(train_constant)
#train_constant.isnull().sum()

In [ ]:
#Code by Parul Pandey  https://www.kaggle.com/parulpandey/a-guide-to-handling-missing-values-in-python


from sklearn.impute import SimpleImputer
df_most_frequent = df.copy()
#setting strategy to 'mean' to impute by the mean
mean_imputer = SimpleImputer(strategy='most_frequent')# strategy can also be mean or median 
df_most_frequent.iloc[:,:] = mean_imputer.fit_transform(df_most_frequent)

In [ ]:
df_most_frequent.isnull().sum()

In [ ]:
from fbprophet import Prophet
df1=df_most_frequent.rename(columns={"Employment_Start_Date": "ds", "PEC_No": "y"})
df1
m = Prophet()
m.fit(df1)

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

#Plot using the Prophet’s plot function.

In [ ]:
fig1 = m.plot(forecast)

#It didn't return an error but I still don't know the meaning of Prophet charts

"What is the dark blue area? Is it the uncertainty interval? What is the light blue area? Why do I see the black dots on the plot?"

Is employement level increasing?? What's the meaning of the black doodle??

#Prophet plot explained - By Bartosz Mikulski


"When I looked at a plot generated by Prophet for the first time, I was lost. I started looking for the explanation in the documentation. There were none. Great, so what now? I googled it. Now I was sure that I was the only person who did not understand the plot because I could not find anything (not even a StackOverflow question or someone asking for an explanation)."

"I had two options. I could either give up or start digging in the source code. Fortunately, when you look at the source code of the Prophet plot function, everything starts to be obvious and easy."

"I cannot spot the individual data points. All I have is a weird broad blue line. It is not an error! It looks like this because there are many data points and they get plotted close to each other. That observation is going to be important later"

"The black dots indicate the data points used to train the model. The next line plots the prediction.Finally, it draws the grid and the label axis:"

https://www.mikulskibartosz.name/prophet-plot-explained/

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
#Code by attollos https://www.kaggle.com/attollos/time-series-forecast-example-with-prophet/

from fbprophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode()

fig = plot_plotly(m, forecast)  # This returns a plotly Figure
py.iplot(fig)

In [ ]:
#Code by Mysterious Ben https://www.kaggle.com/myster/eda-prophet-winning-solution-3-0

_ = pd.pivot_table(df, values='PEC_No', index='Employment_Start_Date').plot(style='-o', title="Pakistan Engineers Employment")
plt.xticks(rotation=45);

#It seems that employment is stable by the end of 2019 till 2020. Though we're in 2021. That information is not useful .